学习目标

● 掌握网络爬虫爬取数据的基本流程

● 掌握requests、BeautifulSoup、chardet等库的使用方法

● 能够完成从爬取到存取的完整流程，自己独立开发简单爬虫

在互联网高速发展的今天，互联网存储了海量有价值的数据，要想 将数据的价值发挥出来，必须进行数据分析，而这一切的起点就是数据 的采集。面对如此多的数据，人工采集显然已经不太现实，那么如何高 效获取这些数据呢？答案就是网络爬虫。

## 数据获取


### 爬虫简介

什么是网络爬虫呢？

**网络爬虫（web crawler）**，也被称为**网络蜘蛛 （web spider）**，是在万维网浏览网页并按照一定规则提取信息的脚本 或者程序。一般浏览网页时，流程如图所示。

![](https://tva1.sinaimg.cn/large/007S8ZIlly1gh5ke12xi2j31430etjtk.jpg)

而利用网络爬虫爬取信息就是模拟这个过程。用脚本模仿浏览器， 向网站服务器发出浏览网页内容的请求，在服务器检验成功后，返回网 页的信息，然后解析网页并提取需要的数据，最后将提取得到的数据保 存即可。关于模拟过程的解释如下。

● 怎样**发起请求**

使用**requests**库来发起请求。

● 服务器**为什么要检验请求**

大量的爬虫请求会造成服务器压力过大，可能使得网页响应速度变 慢，影响网站的正常运行。所以网站一般会检验请求头里面的UserAgent （以下简称UA，相当于身份的识别）来判断发起请求的是不是机 器人，而我们可以通过自己设置 UA 来进行简单伪装。也有些网站设置 有robots.txt来声明对爬虫的限制，例如www.baidu.com/robots.txt

● 怎样**解析**网页并提取数据

这里使用BeautifulSoup库和正则表达式来解析网页并提取数据。

● 怎样**保存**提取的内容

可以根据数据格式的不同将内容保存在TXT、CSV、XLSX、JSON 等文件中，对于数据量比较大的内容，可以选择存入数据库。



注意：有些网站设置robots.txt来声明对爬虫的限制，一般情况下， 我们应当遵守此规则。关于robots.txt的知识，这里仅作简单介绍，详情 请参考维基百科或者 http://www.robotstxt.org/ 之后我们还会简单介绍 如何遵守robots.txt进行数据获取。此外，本书所有爬虫代码示例均为学 习交流之便。作为一名合格的互联网公民，希望读者在开发企业级爬虫 获取数据时仔细阅读相关网站的robots.txt，合理采集数据，切勿对网站 造成过载等不良影响。


### 数据抓取实战

#### 请求网页数据

1. 发起请求


In [12]:
import requests

url = 'https://www.douban.com'

data = requests.get(url)

print(data.request.headers)
# print(data.text)

print(type(data)) # https://requests.readthedocs.io/en/master/_modules/requests/models/

# https://stackoverflow.com/questions/1006169/how-do-i-look-inside-a-python-object
# type()
# dir()
# id()
# getattr()
# hasattr()
# globals()
# locals()
# callable()
print(dir(data))


print(data.text)

print(data.status_code)

{'User-Agent': 'python-requests/2.21.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
<class 'requests.models.Response'>
['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']

418


In [13]:
import requests

url = 'https://www.douban.com'

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}

data = requests.get(url, headers = headers)


print(data.request.headers)
# print(data.text)

print(data.text)

print(data.status_code)

{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
<!DOCTYPE HTML>
<html lang="zh-cmn-Hans" class="ua-windows ua-webkit">
<head>
<meta charset="UTF-8">
<meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />
<meta name="description" content="提供图书、电影、音乐唱片的推荐、评论和价格比较，以及城市独特的文化生活。">
<meta name="keywords" content="豆瓣,广播,登陆豆瓣">
<meta property="qc:admins" content="2554215131764752166375" />
<meta property="wb:webmaster" content="375d4a17a4fa24c2" />
<meta name="mobile-agent" content="format=html5; url=https://m.douban.com">
<title>豆瓣</title>
<script>
function set_cookie(t,e,o,n){var i,a,r=new Date;r.setTime(r.getTime()+24*(e||30)*60*60*1e3),i="; expires="+r.toGMTString();for(a in t)document.cookie=a+"="+t[a]+i+"; domain="+(o||"douban.com")+"; path="+(n||"/")}function get_cookie(t){var e,o,n=t

下面进行简要说明。

● import requests

调入需要的库requests。

● url=＇http://www.douban.com＇

将变量url赋值为豆瓣的网址。

● data=requests.get(url)

利用requests库的get方法，向此URL（即豆瓣首页）发起请求，并 将服务器返回的内容存入变量data。

● print(data.text)

打印服务器返回的内容。从打印内容来看，已经请求成功。


#### 设置UA进行伪装

那么，如何设置UA进行伪装呢？这里介绍一个网址， http://httpbin.org/get， 它会返回一些关于请求头的信息。下面是访问时返回的内容。

In [14]:
import requests

url = 'http://httpbin.org/get'

data = requests.get(url)

print(data.request.headers)
# print(data.text)

print(data.text)

print(data.status_code)

{'User-Agent': 'python-requests/2.21.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.21.0", 
    "X-Amzn-Trace-Id": "Root=1-5f1e969e-283418bc98fa93f4a639a920"
  }, 
  "origin": "175.162.6.164", 
  "url": "http://httpbin.org/get"
}

200


可以看到返回的UA如下。

{'User-Agent': 'python-requests/2.21.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
显然，在没有UA的伪装下，服务器很容易就能识别出对方是一只 网络爬虫的，所以有些网站在发现请求来自网络爬虫时将直接拒绝请 求。为了伪装，可以通过下面的方式设置UA的伪装。

![](https://tva1.sinaimg.cn/large/007S8ZIlly1gh5mvn2llaj30u00v9hdt.jpg)

### 网页解析

在得到网页的内容后，要通过解析从中提取我们想要的信息。在解 析前，要明确爬取的内容，这里假设想要抓取豆瓣推出的新书信息。即 爬取 https://book.douban.com/latest 上面书籍的信息。

先看下面一段代码。

In [16]:
import requests
from bs4 import BeautifulSoup

url = 'https://book.douban.com/latest'

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}

data = requests.get(url, headers = headers)


# print(data.text)

# 解析数据

soup = BeautifulSoup(data.text,'lxml')

print(soup)

<!DOCTYPE html>
<html class="ua-windows ua-webkit book-new-nav" lang="zh-cmn-Hans">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>新书速递</title>
<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\s+|\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.call(e,o)}catch(e){t(o)}},a=/<sc

下面进行简要说明。

● from bs4 import BeautifulSoup

调入要使用的库bs4。

● soup=BeautifulSoup(data.text,＇lxml＇)

将网页数据转化为BeautifulSoup对象，并将这个对象命名为soup。

● print(soup)

打印soup内容。接下来的解析操作，针对BeautifulSoup对象：先检 查元素，观察网页。

![](https://tva1.sinaimg.cn/large/007S8ZIlly1gh5ohm37okj30xl0u0e81.jpg)

In [17]:
# 观察到网页上的书籍按左右分布，按照标签分别提取

books_left = soup.find('ul',{'class':'cover-col-4 clearfix'})
books_left = books_left.find_all('li')
books_left

[<li>
 <a class="cover" href="https://book.douban.com/subject/34988787/">
 <img src="https://img3.doubanio.com/view/subject/s/public/s33650503.jpg"/>
 </a>
 <div class="detail-frame">
 <h2>
 <a href="https://book.douban.com/subject/34988787/">戴上手套擦泪</a>
 </h2>
 <p class="rating">
 <span class="allstar00"></span>
 <span class="font-small color-lightgray">
                                 评价人数不足
                         </span>
 </p>
 <p class="color-gray">
                         [瑞典] 乔纳斯·嘉德尔 / 敦煌文艺出版社 / 2020-7
                     </p>
 <p class="detail">
                         雪夜、表白、初吻。男孩们手牵着手继续走着，一切就此改变。
                     </p>
 </div>
 </li>, <li>
 <a class="cover" href="https://book.douban.com/subject/35093991/">
 <img src="https://img3.doubanio.com/view/subject/s/public/s33678102.jpg"/>
 </a>
 <div class="detail-frame">
 <h2>
 <a href="https://book.douban.com/subject/35093991/">上帝之子</a>
 </h2>
 <p class="rating">
 <span class="allstar00"></span>
 <span class="font-small color

In [19]:
len(books_left)

20

In [20]:

books_right = soup.find('ul',{'class':'cover-col-4 pl20 clearfix'})
books_right = books_right.find_all('li')
books_right

[<li>
 <a class="cover" href="https://book.douban.com/subject/35089547/">
 <img src="https://img3.doubanio.com/view/subject/s/public/s33680613.jpg"/>
 </a>
 <div class="detail-frame">
 <h2>
 <a href="https://book.douban.com/subject/35089547/">土星之环</a>
 </h2>
 <p class="rating">
 <span class="allstar00"></span>
 <span class="font-small color-lightgray">
 </span>
 </p>
 <p class="color-gray">
                         [德] 温弗里德·塞巴尔德 / 广西师范大学出版社 / 2020-8
                     </p>
 <p>
                         一段穿越英格兰东海岸的徒步旅行以及旅途中的所思所感。涉及文学、艺术、历史等多个领域，集体与个人回忆交织，图片与文本相辅，梦境与现实共生。
                     </p>
 </div>
 </li>, <li>
 <a class="cover" href="https://book.douban.com/subject/35062168/">
 <img src="https://img3.doubanio.com/view/subject/s/public/s33679040.jpg"/>
 </a>
 <div class="detail-frame">
 <h2>
 <a href="https://book.douban.com/subject/35062168/">醉钢琴与地下蓝调</a>
 </h2>
 <p class="rating">
 <span class="allstar45"></span>
 <span class="font-small color-lightgray">
                     

这里涉及BeautifulSoup对象的用法，解析如下。

● find 通过观察要提取的标签和此处的写法，

可以发现find是找到网页中 标签为ul、类class为cover-col-4 clearfix的第一个内容。返回的对象拥有 一些易于操作的属性，这是BeautifulSoup赋予它们的，接下来还会解释这些属性。

● find_all 

同样，find_all用于找到所有符合要求的标签内容，返回一个列表。 列表的每一个元素和find返回的对象是一样的，拥有一些功能强大的属性。

下面对代码进行简要说明。

● books_left=soup.find(＇ul＇,{＇class＇:＇cover-col-4 clearfix ＇})books_left=books_left.find_all(＇li＇) 获取左半边书籍列表内容；提取左边列表所有书籍的内容，存储到 book_left。

● books_right=soup.find(＇ul＇,{＇class＇:＇cover-col-4 pl20 clearfix ＇}) books_right=books_right.find_all(＇li＇) 获取右半边书籍列表内容；提取右边列表所有书籍的内容，存储到 book_right。

● books=list(books_left)+list(books_right)

将所有单个的图书信息块，存到一个列表内，方便后续统一操作。 接下来，对每个包含单个图书信息的“信息块”进行统一的提取操作。

In [22]:
books=list(books_left)+list(books_right)

# 对每一个图书区块进行相同操作，获取图书信息

img_urls = []

titles = []

ratings = []

authors = []

details = []

for book in books:
    # 图 书 封面 图 片 uzl 地 址
    img_url = book.find_all('a')[0].find('img') .get('src')
    img_urls.append(img_url)
    # 图 书 标题
    title = book.find_all('a')[1].get_text()
    titles.append(title)
    # print (title)
    # 评价 星 级
    rating = book.find('p', {'class':'rating'}).get_text()
    rating = rating.replace('\n','').replace(' ','')
    ratings.append(rating)
    # 作者 及 出 版 信息
    author = book.find('p', {'class':'color-gray'}).get_text()
    author = author.replace('\n','').replace(' ','')
    authors.append (author)
    # 图 书简 介
    detail = book.find_all('p') [2].get_text()
    detail = detail.replace('\n','').replace(' ','')
    details.append (detail)
    
print(details)

['雪夜、表白、初吻。男孩们手牵着手继续走着，一切就此改变。', '一场大火之后，巴拉德过上了穴居生活，从现代文明中一步步倒着退回荒野，退到世界的边缘。他失去房子，失去名誉，失去社会联结，直至失去生命。', '2025年一个夜晚，15岁的黑人女孩劳伦仰望星空，脑中出现一个词：地球之种。此时，地球自然环境急剧恶化，美国社会和经济危机四伏，新奴隶制度悄然兴起……', '一个中年男子与美术馆的故事，一首彩铅绘就的存在主义图像诗，一组值得所有喜欢独自逛展馆的人细赏的画卷。', '退役间谍吉勒姆在法国过着安逸的隐居生活，但一封密函把他紧急召回伦敦。吉勒姆发现，这次他需要面对的不是敌人，而是昔日同僚的后裔。', '收录法国漫画家夏布特早期发表的四部短篇漫画，均围绕森林、月夜、巫术、解密等元素展开，充满黑色幽默和悬疑色彩。', '埃特加·凯雷特代表作。一个惯于说谎的人有一天竟然遇到了说谎时虚构的人物；一个女孩在男友的嘴里发现一个拉链，拉开后竟出现一个完全不同的男人......', '一位过气的小说家和如今已是成功编剧的大学同学冒着泄露家庭隐私的风险，一意报复文风尖酸刻薄的女记者。小说讽刺了媒体与名人文化之间的贪婪关系。', '理想国版《毛姆短篇小说全集》第四卷', '里多尔菲家族都是小矮人。为了使女儿相信这样的身高才是正常的，他们在庄园里只雇用小矮人，还为她买了一个矮人同伴——但是后来同伴突然长高了。怎么办？', '本书收录了布鲁诺·舒尔茨存世的全部虚构作品：两部短篇小说集《鳄鱼街》《用沙漏做招牌的疗养院》，以及集外的3个短篇，构成了一个个既彼此独立又有内在联系的故事。', '陈楸帆近年创作的科幻中短篇小说集，合称为“异化引擎档案”。从不同个体到集体的切片视角，记录下神秘的未知力量“异化引擎”对人类文明所带来的冲击。', '小说用干净素朴的南方方言，以南货店为背景，勾勒出20世纪70年代末到90年代初极具烟火气的江南城镇生活图景，是乡愁般的“古典中国叙事”在当代的回响。', '一幅没有署名的油画，可能是大师之作，但被专家一句话贬为赝品。主人公跨越时间、地域，根据蛛丝马迹一步步逼近一个多世纪前的真相……', '收录陈映真创作于1967-1982年间的6部中短篇小说，聚焦两次世界大战后台湾饱食、腐败、奢侈、炫丽、幸福的“后街”，反思环境的崩坏、人的伤痕、文化的失据。', '直木奖得主

![](https://tva1.sinaimg.cn/large/007S8ZIlly1gh5psqqmjaj30u00xvh20.jpg)

●＃图书封面图片url地址 

img_url=book.find_all(＇a＇)[0].find(＇img＇).get(＇src＇) img_urls.append(img_url) 可以看出图片地址在此“信息块”的第一个 a 标签内，通过 find_all（＇a＇）找到所有a标签，再通过索引[0]提取第一个a标签的内 容，仔细观察会发现，URL在此a标签下的img标签内。同样的方法，定 位到此img标签。应用find返回对象的get方法，获取src对应的值，即为 要找的URL地址。将此图书的URL加入事先准备好的img_urls列表内， 方便进一步的利用与存取操作。

之后用同样的方法对单个图书的“信息块”提取标题、简介等信息。

● get_text() 此方法可以去除find返回对象内的html标签，返回纯文本。

● replace 由于纯文本有时会包含换行符（\n）和空格等，不易阅读，此处可 使用适当的字符替换，从而美化提取内容。

至此，已经完成了网页信息的提取。此时完整的代码如下，可以打 印提取的内容。

In [24]:
import requests
from bs4 import BeautifulSoup

url = 'https://book.douban.com/latest'

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}

data = requests.get(url, headers = headers)


# print(data.text)

# 解析数据

soup = BeautifulSoup(data.text,'lxml')

# print(soup)

# 观察到网页上的书籍按左右分布，按照标签分别提取

books_left = soup.find('ul',{'class':'cover-col-4 clearfix'})
books_left = books_left.find_all('li')

books_right = soup.find('ul',{'class':'cover-col-4 pl20 clearfix'})
books_right = books_right.find_all('li')

books=list(books_left)+list(books_right)

# 对每一个图书区块进行相同操作，获取图书信息

img_urls = []

titles = []

ratings = []

authors = []

details = []

for book in books:
    # 图 书 封面 图 片 uzl 地 址
    img_url = book.find_all('a')[0].find('img') .get('src')
    img_urls.append(img_url)
    # 图 书 标题
    title = book.find_all('a')[1].get_text()
    titles.append(title)
    # print (title)
    # 评价 星 级
    rating = book.find('p', {'class':'rating'}).get_text()
    rating = rating.replace('\n','').replace(' ','')
    ratings.append(rating)
    # 作者 及 出 版 信息
    author = book.find('p', {'class':'color-gray'}).get_text()
    author = author.replace('\n','').replace(' ','')
    authors.append (author)
    # 图 书简 介
    detail = book.find_all('p') [2].get_text()
    detail = detail.replace('\n','').replace(' ','')
    details.append (detail)
    
    
print ("img urls: ", img_urls)
print ("titles: ", titles)
print ("ratings: ", ratings)
print ("authors: ", authors)
print ("details: ", details)

img urls:  ['https://img3.doubanio.com/view/subject/s/public/s33650503.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33678102.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33673382.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33643223.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33662771.jpg', 'https://img1.doubanio.com/view/subject/s/public/s33664568.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33652573.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33673901.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33668530.jpg', 'https://img1.doubanio.com/view/subject/s/public/s33672769.jpg', 'https://img1.doubanio.com/view/subject/s/public/s33691387.jpg', 'https://img1.doubanio.com/view/subject/s/public/s33658179.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33680423.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33661872.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33663381.jpg', 'https://img3

### 数据的存储

数据的存储方式有很多种，这里将数据存为 CSV 格式，这是一种 非常常见的格式，其他的几种存储方式，将在稍后进行详细介绍。 首先看下面一段代码。

In [25]:
import pandas as pd

result = pd.DataFrame ()
result['img_urls'] = img_urls
result['titles'] = titles
result['ratings'] = ratings
result['authors'] = authors
result ['details'] = details

result.to_csv('result.csv', index=None)

代码解析如下。

● import pandas as pd 

导入pandas库，将其简化记为pd。

● result=pd.DataFrame() 

创建空的DataFrame数据框，便于数据的存储。数据框就是一种数 据存取的格式，类似于Python原生的字典。

● result[＇img_urls＇]=img_urls 

result[＇titles＇]=titles 

result[＇ratings＇]=ratings 

result[＇authors＇]=authors 

result[＇details＇]=details 

将对应数据填充到数据框。

● result.to_csv(＇result.csv＇,index=None) 

调用to_csv方法，将DataFrame直接转化为CSV格式。 注意：我们一般将这种类型的数据存为 CSV 格式，一是因为格式 简单清晰，二是因为大部分第三方库都可以直接导入CSV格式的文件。



至此，已经完成了一个完整的数据爬取的流程，总地来说就是分为 三个步骤：请求并获取网页数据、解析网页提取有价值的数据，以及存 储爬取的数据。

最后，为了提升代码的可读性，可以将上述三个过程抽象成三个函 数，再将其纳入run函数。

In [41]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
# 请求数据
def get_data():
    url = 'https://book.douban.com/latest'
    # headers BMA) SHA]
    headers = {'User-Agent': "Mozilla/5.0 (X11; Ubuntu; Linux x86 64; rv:52.0) Gecko/20100101 Firefox/52.0"}
    data = requests.get(url, headers=headers)
    # print (data. text)
    return data
# 解析数据
def parse_data(data) :
    soup = BeautifulSoup(data.text, 'lxml')
    # print (soup)
    # 观察 到 网 页 上 的 书籍 按 左右 两 边 分 布 ， 按 照 标签 分 别提 取
    books_left = soup.find('ul', {'class':'cover-col-4 clearfix'})
    books_left = books_left.find_all('li')
    books_right = soup.find('ul', {'class':'cover-col-4 pl20 clearfix'})
    books_right = books_right.find_all('li')
    books = list(books_left)+list(books_right)
    
    # 对 每 一 个 图 书 区 块 进行 相同 的 操作 ， 获 取 图 书信 息

    img_urls = []
    titles = []
    ratings = []
    authors = []
    details = []
    for book in books:
        # 图书封面图片URL地址
        img_url = book.find_all('a')[0].find('img').get('src')
        img_urls.append(img_url)
        # 图书标题
        title = book.find_all('a') [1] .get_text ()
        titles.append (title)
        # print (title)
        # 评价星级
        rating = book.find('p', {'class':'rating'}) .get_text()
        rating = rating.replace('\n','').replace(' ','')
        ratings.append (rating)
        # 作者及出版信息
        author = book.find('p', {'class':'color-gray'}) .get_text ()
        author = author.replace('\n','').replace(' ','')
        authors.append (author)
        # 图书简介
        detail = book.find_all('p')[2].get_text()
        detail = detail.replace('\n','').replace(' ','')
        details.append (detail)

    print ("img_urls: ", img_urls)
    print ("titles: ", titles)
    print ("ratings: ", ratings)
    print ("authors: ", authors)
    print ("details: ", details)
    return img_urls,titles,ratings, authors, details
# 存储 数据
def save_data(img_urls,titles,ratings,authors,details) :
    result = pd.DataFrame ()
    result['img_ urls'] = img_urls
    result['titles'] = titles
    result['ratings'] = ratings
    result ['authors'] = authors
    result['details'] = details
    result.to_csv('result.csv', index=None)
# 开始 爬 取
def run():
    data = get_data()
    img_urls,titles,ratings, authors, details = parse_data (data)
    save_data(img_urls,titles,ratings, authors, details)
                               
                               
if __name__ =='__main__':
    run ()

img_urls:  ['https://img3.doubanio.com/view/subject/s/public/s33650503.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33678102.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33673382.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33643223.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33662771.jpg', 'https://img1.doubanio.com/view/subject/s/public/s33664568.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33652573.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33673901.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33668530.jpg', 'https://img1.doubanio.com/view/subject/s/public/s33672769.jpg', 'https://img1.doubanio.com/view/subject/s/public/s33691387.jpg', 'https://img1.doubanio.com/view/subject/s/public/s33658179.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33680423.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33661872.jpg', 'https://img3.doubanio.com/view/subject/s/public/s33663381.jpg', 'https://img3

● 关于if__name__==＇__main__＇：

它作为程序入口，一般在涉及多个文件相 互调用时才起作用。将最后两行换为run（）也可以运行，这里只是遵 循一般性的写法。在本书的第1章也有关于这个用法的详细介绍。 

● 关于函数

函数的关键在于参数的传递，一个函数执行完自己的功能，返回其 得到的结果供下一个函数操作，如此传递下去，直到从原始的网页数据 提取到想要的数据。如果这里读者理解起来比较困难，那么也不必担 心，先放一放，等过一段时间再回头看，就会一目了然了。读者只需要 记住，这里使用函数是为了提升代码可读性和便于日后的维护。

● 关于输出

为了便于观察程序的运行，而不影响程序的效率，可以在函数中适 当打印提示信息。例如在 run 函数开始的时候加上 print（＂开始爬取... ＂）等信息。这点希望读者可以注意下，适当打印信息可以帮助我们调 试程序，相信随着不断的实战练习，大家会认识到这一点。


## 爬虫进阶

以上介绍的网页请求、下载、网页内容的解析、数据的提取与储存就是爬虫的全部流程， 但是在流程的内部还有很多细节值得我们进一步去了解。简单讲，只是 写出一个完整的爬虫、熟悉爬虫运行的流程是远远不够的，在数据获取 的过程中，我们的程序还有很多地方需要改进。

### 异常处理

先看下面这段程序：

In [42]:
import requests

url = "https://www.baidusssss.com" # 不存在的网址

data = requests.get(url)

print(data.text)

ConnectionError: HTTPSConnectionPool(host='www.baidusssss.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7ffd4a997ba8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

这里模拟可能会错误抓取 url 地址然后照常请求的真实情况，故意 请求一个不存在的网页，试试看会发生什么？


可以看到，输出一大段报错，并且程序异常终止。试想，这里只是 单独请求这个网址，有错误可以立即更改，但是在一次请求多个网页 时，例如 1000 个，如果当中有一个网址错了，那么整个程序就会终 止，造成其他大部分正常网页都无法完成请求，这就有些得不偿失了！